In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go

# Analysis of Simulation Benchmarking Experiments

The benchmarking experiments were run on  a Dell XPS-15. The metrics reported in the paper come from these experiments.

In [2]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            

In [3]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "VBN"
    elif(row=="e2e"):
        return "LBC"
    else:
        return row.upper()
def replace_obs(row):
    if(row==0):
        return "No" 
    else: 
        return "Yes"

In [4]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs = all_dfs.reset_index().drop(columns='index')
all_dfs

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
0,01/04/2021/17:06:07,8.478,6.64321,3.96654,No,94.933493,5.066507,VBN,0.5,10
1,01/04/2021/17:07:36,8.315,6.60067,3.97587,No,88.899489,11.100511,VBN,0.5,10
2,01/04/2021/17:09:06,8.507,6.43541,3.98251,No,94.946925,5.053075,VBN,0.5,10
3,01/04/2021/17:10:21,8.885,6.63455,3.96435,No,94.508173,5.491827,VBN,0.5,10
4,01/04/2021/17:11:51,8.353,6.52825,3.98548,No,93.997684,6.002316,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...,...
1817,04/04/2021/17:16:40,26.588,14.33750,5.40917,Yes,33.543118,66.456882,ARS,1.5,25
1818,04/04/2021/17:17:54,31.616,13.74030,6.50090,Yes,20.577657,79.422343,ARS,1.5,25
1819,04/04/2021/17:19:23,27.052,14.65300,5.88859,Yes,27.717371,72.282629,ARS,1.5,25
1820,04/04/2021/17:20:37,29.132,14.57170,5.66425,Yes,25.339281,74.660719,ARS,1.5,25


In [5]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet         mean_et         \
                                                 mean     std    mean    std   
ml_controller speed obstacle_presence runtime                                  
ARS           0.5   No                10       14.729   9.058   6.000  0.251   
                                      25       32.476  18.853  14.048  0.831   
                    Yes               10       14.531  27.378   6.038  0.260   
                                      25       31.742  14.013  14.233  0.782   
              1.0   No                10       16.862   8.472   5.959  0.224   
                                      25       36.092  19.633  14.069  0.478   
                    Yes               10       19.771  26.843   5.922  0.476   
                                      25       34.108  24.763  14.376  0.806   
              1.5   No                10        9.996   4.522   6.274  0.238   
                                      25       28.772  11.828  14.252  0.345   
                    Yes               10       11.667   5.123   6.170  0.276   
                                      25       27.668   1.678  14.327  0.619   
LBC           0.5   No                10        9.027   0.395   6.598  0.092   
                                      25       28.247   0.921  14.498  0.106   
                    Yes               10        9.721   2.803   6.416  0.216   
                                      25       28.719   2.664  14.690  0.212   
              1.0   No                10        9.097   0.494   6.615  0.062   
                                      25       27.852   0.603  14.571  0.201   
                    Yes               10        9.868   1.977   6.350  0.256   
                                      25       31.175  15.402  14.560  2.258   
              1.5   No                10        9.673   1.195   6.675  0.506   
                                      25       27.795   0.755  15.014  1.180   
                    Yes               10       12.841  10.622   6.341  0.612   
                                      25       29.716   5.022  14.541  2.152   
SAC           0.5   No                10       10.367   4.101   6.364  0.185   
                                      25       27.375   1.159  13.986  0.309   
                    Yes               10       11.294   8.555   6.282  0.172   
                                      25       27.660   2.167  14.164  0.469   
              1.0   No                10       11.318   5.625   6.412  0.240   
                                      25       28.148   4.898  13.776  0.503   
                    Yes               10       10.782   3.665   6.143  0.264   
                                      25       28.443   5.628  13.938  0.999   
              1.5   No                10       13.786  17.417   6.291  0.266   
                                      25       28.615   4.537  13.528  1.024   
                    Yes               10       14.328  16.715   6.252  0.305   
                                      25       27.642   1.674  14.030  0.442   
VBN           0.5   No                10        8.497   0.279   6.540  0.176   
                                      25       27.175   0.791  14.444  0.697   
                    Yes               10        8.822   0.703   6.389  0.205   
                                      25       27.496   0.902  14.455  0.286   
              1.0   No                10        8.724   0.377   6.588  0.148   
                                      25       27.392   0.699  14.211  0.221   
                    Yes               10        9.243   1.247   6.309  0.584   
                                      25       27.802   1.884  14.510  0.899   
              1.5   No                10        8.932   0.465   6.554  0.168   
                                      25       27.318   0.937  14.362  0.300   
                    Yes               10        9.552   1.931   6.132  1.022   
                                      25       27

# Generate Latex Tables

In [6]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
#table1 = table1[table1.runtime=='25'].groupby(["ml_controller","runtime"]).mean()
table1 = round(table1.groupby(["ml_controller","runtime"]).mean(),2)
table1
print(table1[table1.columns[2:6]].to_latex())

\begin{tabular}{llrrrr}
\toprule
    &    & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
    &    &    mean &   std &           mean &   std \\
ml\_controller & runtime &         &       &                &       \\
\midrule
ARS & 10 &    6.04 &  0.34 &           4.64 &  0.62 \\
    & 25 &   14.31 &  0.74 &           5.68 &  0.44 \\
LBC & 10 &    6.37 &  0.36 &           4.11 &  0.79 \\
    & 25 &   14.60 &  1.54 &           5.45 &  1.59 \\
SAC & 10 &    6.23 &  0.25 &           4.55 &  0.48 \\
    & 25 &   14.04 &  0.64 &           5.63 &  0.67 \\
VBN & 10 &    6.28 &  0.60 &           4.44 &  1.55 \\
    & 25 &   14.52 &  0.66 &           5.27 &  0.46 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [7]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","runtime"]).mean().reset_index(),2)
print(table1[table1.columns[4:8]].to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
{} &    mean &   std &           mean &   std \\
\midrule
0 &    6.08 &  0.24 &           4.62 &  0.15 \\
1 &   14.12 &  0.55 &           5.66 &  0.17 \\
2 &    6.63 &  0.22 &           3.95 &  0.10 \\
3 &   14.69 &  0.50 &           5.11 &  0.19 \\
4 &    6.36 &  0.23 &           4.48 &  0.46 \\
5 &   13.76 &  0.61 &           5.67 &  0.71 \\
6 &    6.56 &  0.16 &           4.01 &  0.14 \\
7 &   14.34 &  0.41 &           5.10 &  0.14 \\
\bottomrule
\end{tabular}



In [8]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean(),2)
print(table1[table1.columns[6:8]].to_latex())

\begin{tabular}{llrr}
\toprule
    &     & \multicolumn{2}{l}{ml\_usage} \\
    &     &     mean &    std \\
ml\_controller & speed &          &        \\
\midrule
ARS & 0.5 &    66.06 &   9.10 \\
    & 1.0 &    42.43 &   7.98 \\
    & 1.5 &    26.50 &   5.84 \\
LBC & 0.5 &    83.57 &  13.04 \\
    & 1.0 &    55.72 &  24.11 \\
    & 1.5 &    33.44 &  12.51 \\
SAC & 0.5 &    48.38 &  18.29 \\
    & 1.0 &    15.45 &   7.83 \\
    & 1.5 &     8.27 &   3.92 \\
VBN & 0.5 &    78.63 &  12.33 \\
    & 1.0 &    55.55 &  16.53 \\
    & 1.5 &    39.79 &  12.29 \\
\bottomrule
\end{tabular}



In [9]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean().reset_index(),2)
print(table1[table1.columns[8:10]].to_latex())

\begin{tabular}{lrr}
\toprule
{} & \multicolumn{2}{l}{ml\_usage} \\
{} &     mean &    std \\
\midrule
0  &    66.82 &   6.30 \\
1  &    47.88 &   6.68 \\
2  &    32.45 &   5.53 \\
3  &    99.59 &   1.12 \\
4  &    95.90 &   8.66 \\
5  &    37.19 &  17.90 \\
6  &    51.58 &  13.29 \\
7  &    11.82 &   9.07 \\
8  &     8.86 &   4.41 \\
9  &    94.20 &   1.00 \\
10 &    82.10 &   9.91 \\
11 &    55.00 &   8.11 \\
\bottomrule
\end{tabular}



In [10]:
withoutobs = all_dfs[all_dfs.obstacle_presence=="No"]
res = round(withoutobs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
print(res[res.columns[:-2]].to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} \\
    &     &    &   mean &    std &    mean &   std &           mean &   std &     mean &    std \\
ml\_controller & speed & runtime &        &        &         &       &                &       &          &        \\
\midrule
ARS & 0.5 & 10 &  14.73 &   9.06 &    6.00 &  0.25 &           4.78 &  0.07 &    67.71 &   6.37 \\
    &     & 25 &  32.48 &  18.85 &   14.05 &  0.83 &           5.88 &  0.14 &    65.92 &   6.23 \\
    & 1.0 & 10 &  16.86 &   8.47 &    5.96 &  0.22 &           4.99 &  0.13 &    47.27 &   7.30 \\
    &     & 25 &  36.09 &  19.63 &   14.07 &  0.48 &           5.93 &  0.17 &    48.50 &   6.05 \\
    & 1.5 & 10 &  10.00 &   4.52 &    6.27 &  0.24 &           4.10 &  0.26 &    30.63 &   7.53 \\
    &     & 25 &  28.77 &  11.83 &   14.25 &  0.34 &           5.18 &  0.20 &    34.27 &   3.53 

# Analysis of Hardware Experiments

In [16]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","total_time","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                print(file_path)
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs

,time_string,moet,mean_et,avg_iterations,ml_usage,sc_usage,ml_controller,speed,runtime
0,03/04/2021/13:25:54,10.935,5.78091,4.99117,17.917072,82.082928,VBN,0.5,10
1,03/04/2021/13:27:15,11.029,6.30026,4.55825,27.399794,72.600206,VBN,0.5,10
2,03/04/2021/13:28:57,13.587,6.11045,4.89243,26.247901,73.752099,VBN,0.5,10
3,03/04/2021/13:30:38,24.368,5.69501,4.36828,20.250907,79.749093,VBN,0.5,10
4,03/04/2021/13:32:13,12.129,5.33000,4.88028,17.869757,82.130243,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...
0,03/04/2021/13:56:42,24.684,11.01770,5.55328,19.607765,80.392235,ARS,1.0,25
1,03/04/2021/13:58:20,23.122,11.38650,5.33877,15.694671,84.305329,ARS,1.0,25
2,03/04/2021/13:59:56,24.557,11.03970,5.27128,29.077913,70.922087,ARS,1.0,25
3,03/04/2021/14:01:38,25.477,11.30330,5.77496,26.734366,73.265634,ARS,1.0,25


In [19]:
res = round(hw_dfs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
res[res.columns[:-2]]
print(res[res.columns[:-2]].to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} \\
    &     &    &   mean &   std &    mean &   std &           mean &   std &     mean &    std \\
ml\_controller & speed & runtime &        &       &         &       &                &       &          &        \\
\midrule
ARS & 0.5 & 10 &  13.81 &  6.39 &    4.88 &  0.26 &           4.90 &  0.71 &    20.22 &   5.30 \\
    &     & 25 &  23.74 &  0.64 &   11.27 &  0.66 &           5.95 &  0.63 &    33.94 &   8.15 \\
    & 1.0 & 10 &  11.21 &  0.87 &    5.09 &  0.42 &           4.64 &  0.46 &    34.24 &  36.91 \\
    &     & 25 &  24.53 &  0.86 &   11.15 &  0.18 &           5.54 &  0.24 &    22.64 &   5.38 \\
LBC & 0.5 & 10 &  13.36 &  2.61 &    6.22 &  0.52 &           4.77 &  0.16 &    21.75 &   6.90 \\
    &     & 25 &  24.82 &  0.99 &   11.34 &  0.64 &           5.37 &  0.19 &    29.15 &   5.57 \\
    &